In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_data_file: Path
    local_data_name: str

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        key_filepath = KEY_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.key = read_yaml(key_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_data_file=config.source_data_file,
            local_data_name=config.local_data_name,
        )

        return data_ingestion_config

In [5]:
import os
from gemInsights import logger
import pandas as pd

In [8]:
# components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def load_data(self):
        data_path = Path(self.config.source_data_file)
        df = pd.read_csv(data_path)
        logger.info(f"loaded the dataframe with shape {df.shape}")

        df.to_csv(os.path.join(self.config.root_dir, self.config.local_data_name))
        logger.info(f"saved the data in {os.path.join(self.config.root_dir, self.config.local_data_name)}")

    def store_data(self):
        """
        code for store the data into the cloud
        """

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_data()
    data_ingestion.store_data()
except Exception as e:
    raise e

[2023-12-20 08:41:01,847: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-20 08:41:01,849: INFO: common: yaml file: gemini_key.json loaded successfully]
[2023-12-20 08:41:01,849: INFO: common: created directory at: artifacts]
[2023-12-20 08:41:01,850: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-20 08:41:01,854: INFO: 2535834730: loaded the dataframe with shape (1000, 18)]
[2023-12-20 08:41:01,863: INFO: 2535834730: saved the data in artifacts/data_ingestion/data.csv]
